In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

In [24]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [25]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [26]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [27]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            #vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.69173, saving model to best_m.h5
 - 4s - loss: 0.7418 - acc: 0.5105 - val_loss: 0.6917 - val_acc: 0.5087
Epoch 2/50
Epoch 00002: val_loss improved from 0.69173 to 0.61490, saving model to best_m.h5
 - 2s - loss: 0.6584 - acc: 0.5447 - val_loss: 0.6149 - val_acc: 0.5012
Epoch 3/50
Epoch 00003: val_loss improved from 0.61490 to 0.50520, saving model to best_m.h5
 - 2s - loss: 0.5990 - acc: 0.6253 - val_loss: 0.5052 - val_acc: 0.7681
Epoch 4/50
Epoch 00004: val_loss improved from 0.50520 to 0.39694, saving model to best_m.h5
 - 2s - loss: 0.5046 - acc: 0.7508 - val_loss: 0.3969 - val_acc: 0.8354
Epoch 5/50
Epoch 00005: val_loss improved from 0.39694 to 0.39260, saving model to best_m.h5
 - 2s - loss: 0.4979 - acc: 0.7483 - val_loss: 0.3926 - val_acc: 0.8454
Epoch 6/50
Epoch 00006: val_loss improved from 0.39260 to 0.32578, saving model to best_m.h5
 - 2s - loss: 0.4598 - acc: 0.7839 - val_loss: 0.3258 - val_acc: 0.8628
Epoch 7/50
Epo

Epoch 11/50
Epoch 00011: val_loss improved from 0.31105 to 0.28530, saving model to best_m.h5
 - 2s - loss: 0.3343 - acc: 0.8583 - val_loss: 0.2853 - val_acc: 0.8753
Epoch 12/50
Epoch 00012: val_loss improved from 0.28530 to 0.26789, saving model to best_m.h5
 - 2s - loss: 0.3287 - acc: 0.8531 - val_loss: 0.2679 - val_acc: 0.8778
Epoch 13/50
Epoch 00013: val_loss did not improve
 - 1s - loss: 0.2819 - acc: 0.8783 - val_loss: 0.2898 - val_acc: 0.8778
Epoch 14/50
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.2736 - acc: 0.8891 - val_loss: 0.2689 - val_acc: 0.8828
Epoch 15/50
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.2856 - acc: 0.8633 - val_loss: 0.3006 - val_acc: 0.8703
Epoch 16/50
Epoch 00016: val_loss improved from 0.26789 to 0.25299, saving model to best_m.h5
 - 2s - loss: 0.2874 - acc: 0.8825 - val_loss: 0.2530 - val_acc: 0.8903
Epoch 17/50
Epoch 00017: val_loss did not improve
 - 1s - loss: 0.2773 - acc: 0.8781 - val_loss: 0.2530 - val_acc: 0.8928
Epoch 18/50
Ep

Epoch 22/50
Epoch 00022: val_loss improved from 0.22488 to 0.21282, saving model to best_m.h5
 - 2s - loss: 0.2798 - acc: 0.8700 - val_loss: 0.2128 - val_acc: 0.9102
Epoch 23/50
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.2926 - acc: 0.8533 - val_loss: 0.2417 - val_acc: 0.8953
Epoch 24/50
Epoch 00024: val_loss did not improve
 - 2s - loss: 0.2819 - acc: 0.8664 - val_loss: 0.2686 - val_acc: 0.8778
Epoch 25/50
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.2772 - acc: 0.8766 - val_loss: 0.2604 - val_acc: 0.8953
Epoch 26/50
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.2795 - acc: 0.8681 - val_loss: 0.2334 - val_acc: 0.8878
Epoch 27/50
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2599 - acc: 0.8706 - val_loss: 0.2151 - val_acc: 0.9002
Epoch 28/50
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.2645 - acc: 0.8783 - val_loss: 0.2412 - val_acc: 0.8903
Epoch 29/50
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2486 - acc: 0.8873 - val_loss: 0.233

Epoch 33/50
Epoch 00033: val_loss did not improve
 - 1s - loss: 0.1937 - acc: 0.9150 - val_loss: 0.2738 - val_acc: 0.8653
Epoch 34/50
Epoch 00034: val_loss did not improve
 - 1s - loss: 0.2461 - acc: 0.8867 - val_loss: 0.2663 - val_acc: 0.8753
Epoch 35/50
Epoch 00035: val_loss did not improve
 - 1s - loss: 0.2460 - acc: 0.8848 - val_loss: 0.2756 - val_acc: 0.8803
Epoch 36/50
Epoch 00036: val_loss did not improve
 - 1s - loss: 0.2420 - acc: 0.8991 - val_loss: 0.2659 - val_acc: 0.8703
Epoch 37/50
Epoch 00037: val_loss improved from 0.26200 to 0.25962, saving model to best_m.h5
 - 2s - loss: 0.2062 - acc: 0.9133 - val_loss: 0.2596 - val_acc: 0.8678
Epoch 38/50
Epoch 00038: val_loss did not improve
 - 1s - loss: 0.2363 - acc: 0.9017 - val_loss: 0.2636 - val_acc: 0.8703
Epoch 39/50
Epoch 00039: val_loss did not improve
 - 1s - loss: 0.2102 - acc: 0.8992 - val_loss: 0.2649 - val_acc: 0.8678
Epoch 40/50
Epoch 00040: val_loss did not improve
 - 1s - loss: 0.1964 - acc: 0.9050 - val_loss: 0.269

In [28]:
import pickle
with open('../features/cnn_1_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_1_aug1_sub.csv', index=False)
# pre 2508
# new 2239

# pre 15195, fold4
# fold3, batch 64, 15428, 0.275
# fold5, batch 64, 15275, 0.257
# fold5, batch 16, 15460, 0.227
# fold5, batch 64, best acc, 15403, 0.269
# fold4, batch 64, 15437, 0.258
# fold4, batch 64, best acc, 15343, 0.272
# fold4, batch 16, best acc, 15367, 0.229
# fold4, batch 16, rm vertical flip, 15325, 0.226

0.2265637157
         id  is_iceberg
0  5941774d    0.161162
1  4023181e    0.815921
2  b20200e4    0.638001
3  e7f018bb    0.991685
4  4371c8c3    0.867758
